In [2]:
import numpy as np
import torch

In [3]:
from pathlib import Path

In [4]:
import os

In [5]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import NearestNeighbors

In [6]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# https://plotly.com/python/creating-and-updating-figures/
import plotly.graph_objects as go

from mpl_toolkits import mplot3d

In [7]:
import sys
#sys.path.append('../') # now can access entire repository, (important when running locally)

# from orbit_datasets import neworbits, versatileorbits, staticorbits
from ldcl.data import neworbits, versatileorbits, staticorbits

In [8]:
from ldcl.data import physics

In [9]:
train_orbits_dataset, folder = physics.get_dataset("variance_orbit_config.json", "../saved_datasets")
print(folder)

..\saved_datasets\20221213-4\


In [10]:
train_orbits_dataset.data.shape

(1000, 10, 4)

In [11]:
#orbits_dataset = neworbits.OrbitsDataset()
orbits_loader = torch.utils.data.DataLoader(
    dataset = train_orbits_dataset,
    shuffle = True,
    batch_size = 1,
)

In [12]:
def getRepresentations(encoder_location):
    branch_encoder = torch.load(encoder_location, map_location=torch.device('cpu'))
    branch_encoder.eval()
    
    encoder_outputs_list = []
    target_values = []

    for it, (input1, input2, y) in enumerate(orbits_loader):
        predicted_representation = branch_encoder(input1.float()).detach().numpy()[0]
        encoder_outputs_list.append(predicted_representation)

        #append conserved quantities to the end of the representation for plotting, y = (1, )
        #[2=phi0,3=energy,4=angular_momentum] discard [0=eccentricity, 1=semimajor_axis]
        target_values.append(np.array([y['phi0'].detach().numpy().flatten()[0],
                                       y['H'].detach().numpy().flatten()[0],
                                       y['L'].detach().numpy().flatten()[0]
                                      ]))
    encoder_outputs = np.vstack(encoder_outputs_list)
    target_values = np.vstack(target_values)

    phi0_c_values = target_values[:,0]
    energy_c_values = target_values[:,1]
    angular_momentum_c_values = target_values[:,2]
    
    return encoder_outputs, [phi0_c_values, energy_c_values, angular_momentum_c_values]

In [17]:
ENCODER_PATH = Path("saved_models/rmseNCE_3D/final_encoder.pt")
encoder_outputs, conserved_quantities = getRepresentations(ENCODER_PATH)
annotations = [f'phi_0: {p:.3f}, H: {h:.3f}, L: {l:.3f}' for (p,h,l) in zip(*conserved_quantities)]
#print(annotations)
#print(len(conserved_quantities))

In [43]:
def getAllReps(encoder_location, data):
    branch_encoder = torch.load(encoder_location, map_location=torch.device('cpu'))
    branch_encoder.eval()
    
    rep_list = []
    
    for orbit in data:
        #print(orbit.shape)
        predicted = branch_encoder(torch.from_numpy(orbit).float()).detach().numpy()
        rep_list.append(predicted)
    
    rep_outputs = np.stack(rep_list)
    #print(rep_outputs.shape)
    #print(data.dtype)
    
    return rep_outputs
    #predictions = branch_encoder(data)

In [45]:
reps = getAllReps(ENCODER_PATH, train_orbits_dataset.data)
print(reps.shape)

(1000, 10, 3)


In [47]:
diff = reps[:, None, :, :] - reps[:,:,None,:]
print(diff.shape)

(1000, 10, 10, 3)


In [49]:
dist = np.linalg.norm(diff, axis = -1)
print(dist.shape)

(1000, 10, 10)


In [50]:
print(dist[0])

[[0.         0.35812873 0.23275387 0.19955172 0.17220072 0.05948098
  0.22461222 0.25387096 0.24438898 0.24700321]
 [0.35812873 0.         0.559187   0.5387932  0.49472842 0.30208877
  0.5608667  0.58459514 0.5716494  0.5659848 ]
 [0.23275387 0.559187   0.         0.04461134 0.0795669  0.28905088
  0.02724911 0.05963776 0.0557178  0.04738912]
 [0.19955172 0.5387932  0.04461134 0.         0.07184099 0.2575823
  0.0256489  0.06600679 0.06187892 0.06821407]
 [0.17220072 0.49472842 0.0795669  0.07184099 0.         0.22651471
  0.08409564 0.12988728 0.12244345 0.11554262]
 [0.05948098 0.30208877 0.28905088 0.2575823  0.22651471 0.
  0.2824235  0.31007305 0.2998499  0.30129555]
 [0.22461222 0.5608667  0.02724911 0.0256489  0.08409564 0.2824235
  0.         0.05169411 0.05085481 0.05541423]
 [0.25387096 0.58459514 0.05963776 0.06600679 0.12988728 0.31007305
  0.05169411 0.         0.01471887 0.03558924]
 [0.24438898 0.5716494  0.0557178  0.06187892 0.12244345 0.2998499
  0.05085481 0.01471887

In [51]:
print(np.sum(dist)/90000)

0.15617680121527777


In [54]:
print(np.mean(dist))

0.14055912
